# Train Keras Model

In [1]:
import os
from glob import glob
import numpy as np
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import TensorBoard
from keras import backend as K

import datetime

/home/dries/.conda/envs/tf3.5/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/generated_train'
validation_data_dir = 'data/generated_val'
#nb_train_samples = 2000
#nb_validation_samples = 800
epochs = 20
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
metrics=['accuracy'])

In [4]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
class_mode='binary')

Found 2728 images belonging to 2 classes.


In [5]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
class_mode='binary')

Found 869 images belonging to 2 classes.


In [6]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(datetime.datetime.now()), histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [7]:
nb_train_samples = 2728
nb_validation_samples = 869
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=[tensorboard],
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('first_try.h5')
from keras.models import load_model

Epoch 1/20
85/85 [==============================] - 25s 299ms/step - loss: 0.7050 - acc: 0.4927 - val_loss: 0.6918 - val_acc: 0.5266
Epoch 2/20
85/85 [==============================] - 23s 265ms/step - loss: 0.6938 - acc: 0.5334 - val_loss: 0.6747 - val_acc: 0.6238
Epoch 3/20
85/85 [==============================] - 22s 260ms/step - loss: 0.6769 - acc: 0.6026 - val_loss: 0.6261 - val_acc: 0.6956
Epoch 4/20
85/85 [==============================] - 22s 260ms/step - loss: 0.6492 - acc: 0.6607 - val_loss: 0.5843 - val_acc: 0.7373
Epoch 5/20
85/85 [==============================] - 22s 259ms/step - loss: 0.5943 - acc: 0.7143 - val_loss: 0.4987 - val_acc: 0.8113
Epoch 6/20
85/85 [==============================] - 22s 261ms/step - loss: 0.5116 - acc: 0.7750 - val_loss: 0.4029 - val_acc: 0.8611
Epoch 7/20
85/85 [==============================] - 23s 268ms/step - loss: 0.4094 - acc: 0.8335 - val_loss: 0.2836 - val_acc: 0.9144
Epoch 8/20
85/85 [==============================] - 23s 275ms/step - 

In [8]:
from keras.preprocessing import image

data_path = './data'
folder_name = 'test'
ext = '*.jpg'

path = os.path.join(data_path, folder_name, ext)
files = sorted(glob(path))

for f in files:
    
    img = image.load_img(f, target_size=(img_height, img_width))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    x = x / 255.0
    
    _preds = model.predict_classes(x, verbose=0)
    print('{} - {}'.format(f,_preds))
    


./data/test/1.jpg - [[1]]
./data/test/10.jpg - [[1]]
./data/test/11.jpg - [[1]]
./data/test/12.jpg - [[1]]
./data/test/13.jpg - [[1]]
./data/test/14.jpg - [[1]]
./data/test/15.jpg - [[1]]
./data/test/16.jpg - [[1]]
./data/test/17.jpg - [[0]]
./data/test/18.jpg - [[0]]
./data/test/19.jpg - [[0]]
./data/test/2.jpg - [[1]]
./data/test/20.jpg - [[0]]
./data/test/21.jpg - [[0]]
./data/test/22.jpg - [[0]]
./data/test/23.jpg - [[0]]
./data/test/24.jpg - [[0]]
./data/test/25.jpg - [[0]]
./data/test/26.jpg - [[0]]
./data/test/27.jpg - [[1]]
./data/test/28.jpg - [[1]]
./data/test/29.jpg - [[1]]
./data/test/3.jpg - [[0]]
./data/test/30.jpg - [[1]]
./data/test/31.jpg - [[1]]
./data/test/4.jpg - [[0]]
./data/test/5.jpg - [[0]]
./data/test/6.jpg - [[0]]
./data/test/7.jpg - [[0]]
./data/test/8.jpg - [[0]]
./data/test/9.jpg - [[1]]
